In [19]:
import datetime
import os
import subprocess

import pandas as pd

In [1]:
DATETIME_PARSER = '%Y-%m-%dT%H:%MZ'

In [20]:
mass_root = "moose:/adhoc/projects/radar_archive/data/comp/products/composites/"
fname_mass_template = "{dt.year:04d}{dt.month:02d}{dt.day:02d}.tar"
save_path = "/scratch/dkillick/precip_rediagnosis/radar"

mass_cmd_template = "moo get {args} {src_paths} {dest_path}"
unzip_cmd_template = "gunzip {root}/{files}"
untar_cmd_template = 'tar -xf {path} --directory {dest_root}'

event_start = "2020-02-14T18:00Z"
event_end = "2020-02-17T18:00Z"
date_range = [
    datetime.datetime.strptime(event_start, DATETIME_PARSER),
    datetime.datetime.strptime(event_end, DATETIME_PARSER)
]
archive_time_chunk = 24

In [10]:
def calc_dates_list(start_datetime, end_datetime, delta_hours):
    dates_to_extract = list(pd.date_range(
        start=start_datetime,
        end=end_datetime - datetime.timedelta(seconds=1),
        freq=datetime.timedelta(
            hours=delta_hours)).to_pydatetime())
    return dates_to_extract

In [12]:
dates_to_extract = calc_dates_list(
    datetime.datetime(date_range[0].year,
                      date_range[0].month,
                      date_range[0].day, 0, 0),
    datetime.datetime(date_range[1].year,
                      date_range[1].month,
                      date_range[1].day, 23, 59),
    archive_time_chunk
)
dates_to_extract

[datetime.datetime(2020, 2, 14, 0, 0),
 datetime.datetime(2020, 2, 15, 0, 0),
 datetime.datetime(2020, 2, 16, 0, 0),
 datetime.datetime(2020, 2, 17, 0, 0)]

In [15]:
mass_paths = [os.path.join(mass_root, fname_mass_template.format(dt=dt)) for dt in dates_to_extract]
mass_paths

['moose:/adhoc/projects/radar_archive/data/comp/products/composites/20200214.tar',
 'moose:/adhoc/projects/radar_archive/data/comp/products/composites/20200215.tar',
 'moose:/adhoc/projects/radar_archive/data/comp/products/composites/20200216.tar',
 'moose:/adhoc/projects/radar_archive/data/comp/products/composites/20200217.tar']

In [18]:
mass_radar_get_cmd = mass_cmd_template.format(
    src_paths=mass_paths[0],
    dest_path=save_path,
    args='-f',
)
mass_radar_get_cmd

'moo get -f moose:/adhoc/projects/radar_archive/data/comp/products/composites/20200214.tar /scratch/dkillick/precip_rediagnosis/radar'

In [22]:
op = subprocess.check_output(mass_radar_get_cmd, shell=True)
op

b'### get, command-id=1445537681, estimated-cost=7497963520byte(s), files=1, media=0\n'